<a href="https://colab.research.google.com/github/matidesalegn/Improved-detection-of-fraud-cases-in-e-commerce-and-bank-transactions/blob/task-2/models/model_building_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation

### Load Processed Data
### First, let's load the processed data for both datasets:

### Processed Fraud Data with Country: ../data/processed/processed_fraud_data_with_country.csv
### Processed Credit Card Data: ../data/processed/processed_credit_card_data.csv

In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report, accuracy_score
!pip install mlflow
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer  # Import SimpleImputer
import numpy as np

# Load processed data
fraud_data = pd.read_csv('processed_fraud_data_with_country.csv')
credit_card_data = pd.read_csv('processed_credit_card_data.csv')

# Ensure datetime columns are correctly parsed
fraud_data['signup_time'] = pd.to_datetime(fraud_data['signup_time'])
fraud_data['purchase_time'] = pd.to_datetime(fraud_data['purchase_time'])

# Feature Engineering for fraud_data
# Extract numerical features from datetime columns
fraud_data['signup_hour'] = fraud_data['signup_time'].dt.hour
fraud_data['signup_day'] = fraud_data['signup_time'].dt.day
fraud_data['signup_month'] = fraud_data['signup_time'].dt.month
fraud_data['signup_year'] = fraud_data['signup_time'].dt.year

fraud_data['purchase_hour'] = fraud_data['purchase_time'].dt.hour
fraud_data['purchase_day'] = fraud_data['purchase_time'].dt.day
fraud_data['purchase_month'] = fraud_data['purchase_time'].dt.month
fraud_data['purchase_year'] = fraud_data['purchase_time'].dt.year

# Drop original datetime columns
fraud_data = fraud_data.drop(columns=['signup_time', 'purchase_time'])

# Identify categorical columns
categorical_columns = fraud_data.select_dtypes(include=['object']).columns.tolist()

# Encode categorical features
fraud_data = pd.get_dummies(fraud_data, columns=categorical_columns, drop_first=True)

# Check the structure and columns of the datasets
print("Fraud Data Columns:")
print(fraud_data.columns)
print("\nCredit Card Data Columns:")
print(credit_card_data.columns)

# Feature and target separation for fraud detection data
X_fraud = fraud_data.drop(columns=['class'])  # Features
y_fraud = fraud_data['class']                # Target

# Feature and target separation for credit card data
X_credit = credit_card_data.drop(columns=['Class'])  # Features
y_credit = credit_card_data['Class']                # Target

# Handle missing values in features
imputer = SimpleImputer(strategy='mean')
X_fraud = imputer.fit_transform(X_fraud)
X_credit = imputer.fit_transform(X_credit)

# Handle missing values in target variables
y_fraud = y_fraud.fillna(y_fraud.mode()[0])
y_credit = y_credit.fillna(y_credit.mode()[0])

# Train-test split for fraud data
X_train_fraud, X_test_fraud, y_train_fraud, y_test_fraud = train_test_split(X_fraud, y_fraud, test_size=0.2, random_state=42)

# Train-test split for credit card data
X_train_credit, X_test_credit, y_train_credit, y_test_credit = train_test_split(X_credit, y_credit, test_size=0.2, random_state=42)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.9/202.9 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

<ipython-input-1-7b4618c94eab>:17: DtypeWarning: Columns (92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202) have mixed types. Specify dtype option on import or set low_memory=False.
  fraud_data = pd.read_csv('processed_fraud_data_with_country.csv')


Fraud Data Columns:
Index(['user_id', 'purchase_value', 'age', 'ip_address', 'class',
       'ip_address_int', 'transaction_count', 'time_since_signup',
       'transaction_velocity', 'transaction_time_diff',
       ...
       'country_Turkmenistan_True', 'country_Ukraine_True',
       'country_United Arab Emirates_True', 'country_United Kingdom_True',
       'country_United States_True', 'country_Unknown_True',
       'country_Uruguay_True', 'country_Uzbekistan_True',
       'country_Venezuela_True', 'country_Viet Nam_True'],
      dtype='object', length=8571)

Credit Card Data Columns:
Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')


# Model Selection, Training, and Evaluation

### We'll use several models to compare their performance:

### Logistic Regression
### Decision Tree
### Random Forest
### Gradient Boosting
### Multi-Layer Perceptron (MLP)
### Convolutional Neural Network (CNN) (for credit card data)
### Recurrent Neural Network (RNN) or LSTM (for fraud detection data)

### Model Training and Evaluation
### Train and Evaluate Models
### Here's how you might train and evaluate models for the fraud detection dataset (X_train_fraud, y_train_fraud, X_test_fraud, y_test_fraud). Repeat a similar process for the credit card dataset.

In [2]:
# Initialize models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'Decision Tree': DecisionTreeClassifier(),
    'Random Forest': RandomForestClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'MLP': MLPClassifier(max_iter=1000)
}

# Train and evaluate models for fraud detection data
for name, model in models.items():
    with mlflow.start_run(run_name=f'{name} on Fraud Data'):
        model.fit(X_train_fraud, y_train_fraud)
        y_pred = model.predict(X_test_fraud)
        accuracy = accuracy_score(y_test_fraud, y_pred)

        # Log parameters and metrics
        mlflow.log_params({
            'model': name,
            'dataset': 'Fraud Data',
            'test_size': 0.2,
            'random_state': 42
        })
        mlflow.log_metric('accuracy', accuracy)

        # Save the model
        mlflow.sklearn.log_model(model, f'{name}_model')

        # Print and log classification report
        print(f"Model: {name}")
        print(f"Accuracy: {accuracy}")
        print(classification_report(y_test_fraud, y_pred))
        print("="*60)

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. U

Model: Logistic Regression
Accuracy: 0.9105263157894737
              precision    recall  f1-score   support

           0       0.91      1.00      0.95      1557
           1       0.00      0.00      0.00       153

    accuracy                           0.91      1710
   macro avg       0.46      0.50      0.48      1710
weighted avg       0.83      0.91      0.87      1710

Model: Decision Tree
Accuracy: 0.9514619883040936
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1557
           1       0.99      0.46      0.63       153

    accuracy                           0.95      1710
   macro avg       0.97      0.73      0.80      1710
weighted avg       0.95      0.95      0.94      1710

Model: Random Forest
Accuracy: 0.952046783625731
              precision    recall  f1-score   support

           0       0.95      1.00      0.97      1557
           1       1.00      0.46      0.63       153

    accuracy               

# MLOps Steps
### Versioning and Experiment Tracking with MLflow
### For versioning and experiment tracking, we'll integrate MLflow. Ensure you have MLflow installed (pip install mlflow) and set up in your environment.

In [3]:
# Train and evaluate models for credit card data
for name, model in models.items():
    with mlflow.start_run(run_name=f'{name} on Credit Card Data'):
        model.fit(X_train_credit, y_train_credit)
        y_pred = model.predict(X_test_credit)
        accuracy = accuracy_score(y_test_credit, y_pred)

        # Log parameters and metrics
        mlflow.log_params({
            'model': name,
            'dataset': 'Credit Card Data',
            'test_size': 0.2,
            'random_state': 42
        })
        mlflow.log_metric('accuracy', accuracy)

        # Save the model
        mlflow.sklearn.log_model(model, f'{name}_model')

        # Print and log classification report
        print(f"Model: {name}")
        print(f"Accuracy: {accuracy}")
        print(classification_report(y_test_credit, y_pred))
        print("="*60)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model: Logistic Regression
Accuracy: 0.9972533760585947
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4349
         1.0       0.79      0.55      0.65        20

    accuracy                           1.00      4369
   macro avg       0.89      0.77      0.82      4369
weighted avg       1.00      1.00      1.00      4369

Model: Decision Tree
Accuracy: 0.9988555733577478
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4349
         1.0       0.89      0.85      0.87        20

    accuracy                           1.00      4369
   macro avg       0.95      0.92      0.94      4369
weighted avg       1.00      1.00      1.00      4369

Model: Random Forest
Accuracy: 0.9990844586861982
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      4349
         1.0       1.00      0.80      0.89        20

    accuracy              